In [1]:
!pip install -q duckdb pandas openai kaggle

In [10]:
from google.colab import files
import os

# Upload the CSV file directly
print("Please upload the Myntra dataset CSV file (e.g., myntra_ecommerce_product_data_november_2023.csv):")
uploaded = files.upload()

# Get the uploaded filename (assuming one file)
CSV_FILE = list(uploaded.keys())[0]
print(f"Uploaded file: {CSV_FILE}")

Please upload the Myntra dataset CSV file (e.g., myntra_ecommerce_product_data_november_2023.csv):


Saving Myntra-Ecommerce__20231101_20231130_sample.csv to Myntra-Ecommerce__20231101_20231130_sample (1).csv
Uploaded file: Myntra-Ecommerce__20231101_20231130_sample (1).csv


In [16]:
import duckdb
import pandas as pd
from openai import OpenAI

# Connect to DuckDB (in-memory)
con = duckdb.connect(':memory:')

# Load CSV into table named 'products'
con.execute(f"""
    CREATE OR REPLACE TABLE products AS
    SELECT * FROM read_csv_auto('{CSV_FILE}', header=true, ignore_errors=true)
""")

# Get table schema as string for LLM prompts
schema = con.execute("DESCRIBE products").fetchdf().to_string(index=False)
print("Table Schema:\n", schema)

# Sample data for context
sample_data = con.execute("SELECT * FROM products LIMIT 5").fetchdf().to_string(index=False)
print("\nSample Data:\n", sample_data)

# Initialize OpenAI client
client = OpenAI(api_key="MY_OPEN_AI_KEY")  # Replace with your key or use: from google.colab import userdata; client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

Table Schema:
             column_name column_type null  key default extra
                Uniq Id     VARCHAR  YES None    None  None
        Crawl Timestamp     VARCHAR  YES None    None  None
                Pageurl     VARCHAR  YES None    None  None
                Pdp Url     VARCHAR  YES None    None  None
             Product Id      BIGINT  YES None    None  None
           Product Name     VARCHAR  YES None    None  None
             List Price      BIGINT  YES None    None  None
             Sale Price      BIGINT  YES None    None  None
    Discount Percentage      DOUBLE  YES None    None  None
                  Brand     VARCHAR  YES None    None  None
           Product Rank     VARCHAR  YES None    None  None
                   Size      BIGINT  YES None    None  None
           Availability     BOOLEAN  YES None    None  None
    Product Description     VARCHAR  YES None    None  None
                 Seller     VARCHAR  YES None    None  None
           Category Url  

In [17]:
def generate_sql(query: str, schema: str, sample_data: str) -> str:
    """Use LLM to generate SQL from natural language query."""
    prompt = f"""
    You are a SQL expert for DuckDB. The table name is 'products'. Given this table schema:
    {schema}

    And sample data:
    {sample_data}

    Generate a valid DuckDB SQL query to answer: '{query}'

    - Use SELECT statements.
    - Always use the table name 'products'.
    - Use EXACT column names from the schema (e.g., 'Brand', 'Product_Id', 'List_Price', not 'Product Id' or 'price').
    - Do NOT use spaces in column names unless explicitly shown in the schema (e.g., use 'Product_Id' not 'Product Id').
    - For aggregations, use COUNT, AVG, MIN, MAX, etc.
    - Limit results to 20 rows if listing items.
    - If unsure about column names, choose the closest match from the schema and note it in a comment (e.g., -- Assuming 'Brand' for brand name).
    - Return only the SQL query, no explanations or backticks.
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Or "gpt-4o" for better accuracy
        messages=[{"role": "system", "content": "You are a helpful SQL generator."},
                  {"role": "user", "content": prompt}],
        temperature=0
    )
    sql = response.choices[0].message.content.strip()
    return sql

def execute_sql(sql: str) -> tuple[pd.DataFrame, str]:
    """Execute SQL on DuckDB and return results as DataFrame, with detailed error message if any."""
    try:
        result = con.execute(sql).fetchdf()
        return result, ""
    except Exception as e:
        return pd.DataFrame(), f"SQL Execution Error: {str(e)}"

def generate_insights(query: str, data: str, error: str) -> str:
    """Use LLM to generate insights from retrieved data or handle errors."""
    if error:
        prompt = f"""
        You are a data analyst. The user's question was: '{query}'

        An error occurred while retrieving data: {error}

        Explain the issue in simple terms and suggest how to fix it or rephrase the query. If the error mentions a missing column, suggest checking the schema for the correct column name (e.g., 'Product_Id' instead of 'Product Id'). Provide the schema snippet if relevant.
        """
    else:
        prompt = f"""
        You are a data analyst for Myntra e-commerce products.
        Given the user's question: '{query}'

        And the retrieved data:
        {data}

        Provide clear, insightful analysis in natural language.
        - Summarize key findings.
        - Use bullet points or tables if helpful.
        - Suggest 2-3 follow-up questions to explore further.
        """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful data analyst."},
                  {"role": "user", "content": prompt}],
        temperature=0.2
    )
    insights = response.choices[0].message.content.strip()
    return insights

In [19]:
def run_mini_agent():
    """Interactive loop for the custom agent."""
    print("Mini Agent ready! Ask questions about the Myntra dataset (e.g., 'Top 5 brands by average price'). Type 'exit' to stop.")

    while True:
        user_query = input("\nYour query: ").strip()
        if user_query.lower() == 'exit':
            print("Exiting agent.")
            break

        print(f"Processing: {user_query}")

        # Step 1: Generate SQL
        sql = generate_sql(user_query, schema, sample_data)
        print(f"Generated SQL: {sql}")

        # Step 2: Execute SQL (Retrieval)
        result_df, error = execute_sql(sql)
        data_str = result_df.to_string(index=False) if not result_df.empty else "No data found."

        # Step 3: Generate Insights (Augmentation + Generation)
        insights = generate_insights(user_query, data_str, error)
        print("\nInsights:")
        print(insights)

# Start the agent
run_mini_agent()

Mini Agent ready! Ask questions about the Myntra dataset (e.g., 'Top 5 brands by average price'). Type 'exit' to stop.

Your query: What is the average Sale_Price for products in the 'Shirts' category in Sub_Category_3?
Processing: What is the average Sale_Price for products in the 'Shirts' category in Sub_Category_3?
Generated SQL: SELECT AVG(Sale_Price) AS Average_Sale_Price
FROM products
WHERE Category_Url = 'Shirts'
AND Sub_Category_3 IS NOT NULL
AND Sub_Category_3 <> ''
GROUP BY Sub_Category_3;

Insights:
The error message indicates that the column "Category_Url" is not found in the FROM clause, which is causing the SQL query to fail. This means that the query is referencing a column that is not included in the part of the database where it is looking for data.

To fix this issue, you should check the schema of the database to find the correct column name that should be used in the query. In this case, it seems like the correct column name for the category might be different from 